<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-02-ec6e3e9ce321'
PROJECT = 'qwiklabs-gcp-02-ec6e3e9ce321'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [4]:
# Create SQL query using natality data after the year 2000
from google.cloud import bigquery
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [5]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bigquery.Client().query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


hashmonth  num_babies
0  3408502330831153141      323970
1  1088037545023002395      362125
2 -9068386407968572094      330863
3  7186614341837170520      346919
4 -8668300920110565493      355748

Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [6]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) = 3 AND RAND() < 0.0005"
traindf = bigquery.Client().query(trainQuery).to_dataframe()
evaldf = bigquery.Client().query(evalQuery).to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13422 examples in the train dataset and 3345 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [7]:
traindf.head()

weight_pounds  is_male  mother_age  plurality  gestation_weeks  \
0       6.933538    False          26          1             41.0   
1       8.437091    False          30          1             40.0   
2       6.812284    False          14          1             39.0   
3       6.514660     True          30          1             40.0   
4       7.874912    False          22          1             40.0   

             hashmonth  
0  3095933535584005890  
1  3095933535584005890  
2  3095933535584005890  
3  3095933535584005890  
4  3095933535584005890

Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [8]:
# Let's look at a small sample of the training data
traindf.describe()

weight_pounds    mother_age     plurality  gestation_weeks  \
count   13408.000000  13422.000000  13422.000000     13316.000000   
mean        7.215961     27.397631      1.035986        38.617378   
std         1.320645      6.176169      0.197899         2.603783   
min         0.617294     13.000000      1.000000        17.000000   
25%         6.563162     23.000000      1.000000        38.000000   
50%         7.312733     27.000000      1.000000        39.000000   
75%         8.062305     32.000000      1.000000        40.000000   
max        14.250681     49.000000      5.000000        47.000000   

          hashmonth  
count  1.342200e+04  
mean   3.187630e+17  
std    5.217610e+18  
min   -9.183606e+18  
25%   -3.340563e+18  
50%   -3.280124e+17  
75%    4.331750e+18  
max    8.599690e+18

In [9]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [10]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

weight_pounds is_male  mother_age  plurality  gestation_weeks  \
0       6.933538   False          26  Single(1)             41.0   
1       8.437091   False          30  Single(1)             40.0   
2       6.812284   False          14  Single(1)             39.0   
3       6.514660    True          30  Single(1)             40.0   
4       7.874912   False          22  Single(1)             40.0   

             hashmonth  
0  3095933535584005890  
1  3095933535584005890  
2  3095933535584005890  
3  3095933535584005890  
4  3095933535584005890

In [11]:
traindf.tail()

weight_pounds  is_male  mother_age  plurality  gestation_weeks  \
13417       7.749249  Unknown          29  Single(1)             40.0   
13418       7.255413  Unknown          31  Single(1)             39.0   
13419       7.749249  Unknown          30  Single(1)             40.0   
13420       8.562754  Unknown          24  Single(1)             39.0   
13421       6.730713  Unknown          20  Single(1)             40.0   

                hashmonth  
13417 -774501970389208065  
13418 -774501970389208065  
13419 -774501970389208065  
13420 -774501970389208065  
13421 -774501970389208065

In [12]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

weight_pounds    mother_age  gestation_weeks     hashmonth
count   26614.000000  26614.000000     26614.000000  2.661400e+04
mean        7.216000     27.397385        38.619148  3.265063e+17
std         1.317129      6.175885         2.597569  5.211745e+18
min         0.617294     13.000000        17.000000 -9.183606e+18
25%         6.563162     23.000000        38.000000 -3.340563e+18
50%         7.312733     27.000000        39.000000 -3.280124e+17
75%         8.062305     32.000000        40.000000  4.331750e+18
max        14.250681     49.000000        47.000000  8.599690e+18

<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [13]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [14]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

   6634 eval.csv
  26614 train.csv
  33248 total
==> eval.csv <==
9.1822532123,True,27,Single(1),40.0,1088037545023002395
8.062304921339999,True,25,Single(1),39.0,1088037545023002395
8.1240343547,False,41,Single(1),41.0,2246942437170405963
7.2532084198,False,30,Single(1),39.0,1088037545023002395
6.1883756943399995,False,21,Single(1),43.0,-6141045177192779423
8.437090766739999,False,28,Single(1),40.0,3182182455926341111
6.6359140862,False,22,Single(1),40.0,8904940584331855459
5.8753192823,True,24,Single(1),37.0,6392072535155213407
6.75055446244,False,26,Single(1),41.0,8904940584331855459
8.313631900019999,True,34,Single(1),39.0,1569531340167098963

==> train.csv <==
6.9335381399,False,26,Single(1),41.0,3095933535584005890
8.437090766739999,False,30,Single(1),40.0,3095933535584005890
6.8122838958,False,14,Single(1),39.0,3095933535584005890
6.5146598420999995,True,30,Single(1),40.0,3095933535584005890
7.87491199864,False,22,Single(1),40.0,3095933535584005890
6.87621795178,False,22,Single(

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License